# 导入第三方包

In [1]:
import os#没用
import gc#没用
import math #没用

import pandas as pd
import numpy as np

import lightgbm as lgb
import xgboost as xgb #没有用


from sklearn.preprocessing import MinMaxScaler


from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss #没有用
from sklearn.model_selection import train_test_split #没有用
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt #没有用
import time 
import warnings
warnings.filterwarnings('ignore')

# 读取数据

In [34]:
train = pd.read_csv('train.csv') #读取训练集
test=pd.read_csv('testA.csv')#读取测试集


# 数据预处理

In [4]:
#降低数字精度，压缩文件
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
# 数据预处理
train_list = []

for items in train.values: #dataframe.values，把dataframe转换为numpy数组
    #把第二列的值按照','分成列表，并且列表里的元素转换为浮点型，然后第一列和第三列，与第二列组成一个长列表。然后追加到train_list
    train_list.append([items[0]] + [float(i) for i in items[1].split(',')] + [items[2]]) 

train = pd.DataFrame(np.array(train_list)) #把train_list转换成numpy数组，再转换成dataframe
train.columns = ['id'] + ['s_'+str(i) for i in range(len(train_list[0])-2)] + ['label'] #修改列名
train = reduce_mem_usage(train) #调用压缩函数

#原理同上
test_list=[]
for items in test.values:
    test_list.append([items[0]] + [float(i) for i in items[1].split(',')])

test = pd.DataFrame(np.array(test_list))
test.columns = ['id'] + ['s_'+str(i) for i in range(len(test_list[0])-1)]
test = reduce_mem_usage(test)


Memory usage of dataframe is 157.93 MB
Memory usage after optimization is: 39.67 MB
Decreased by 74.9%
Memory usage of dataframe is 31.43 MB
Memory usage after optimization is: 7.90 MB
Decreased by 74.9%


In [35]:
#train.head()

In [36]:
# test.head()

# 训练数据/测试数据准备

In [20]:
x_train = train.drop(['id','label'], axis=1) #axis=1 表示丢弃的是列 ，必须有这个，不然代码报错
y_train = train['label']
x_test=test.drop(['id'], axis=1)

In [37]:
# train.head()

# 模型训练

In [16]:
def abs_sum(y_pre,y_tru): #评价函数
    y_pre=np.array(y_pre)
    y_tru=np.array(y_tru)
    loss=sum(sum(abs(y_pre-y_tru)))
    return loss
    

In [30]:
def cv_model(clf, train_x, train_y, test_x, clf_name): #交叉验证
    folds = 10 #k折交叉验证 。 5改成10，其他条件都不变的情况下，分数从559.6118提升到534.6480 ;k=10，使用的StratifiedKFold，其他不变，分数却变差了
    
    seed = 2021 #随机种子
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)  #StratifiedKFold是针对样本分类不均衡的K折法，比如欺诈电话分类
    test = np.zeros((test_x.shape[0],4))

    cv_scores = []
    
    #OneHotEncoder，心电图类别是名义变量，用哑变量的处理方式。
    #初始化OneHotEncoder实例时，默认sparse参数为True，编码后返回的是一个稀疏矩阵的对象，
    #如果要使用一般要调用toarray()方法转化成array对象。若将sparse参数设置为False，则直接生成array对象，可直接使用。
    onehot_encoder = OneHotEncoder(sparse=False) 
    
     #train_index , valid_index是一个列表，列表里是训练集的序列或者测试局的序列
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        
        #分训练集和验证集，比train_test_split(x, y, test_size=0.333, random_state=0) 好很多 
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
        
        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y) #处理成lgb算法需要的数据格式，训练集
            valid_matrix = clf.Dataset(val_x, label=val_y)#处理成lgb算法需要的数据格式，验证集

            params = {                 #lgb参数
                'boosting_type': 'gbdt',
                'objective': 'multiclass',
                'num_class': 4,
                'num_leaves': 2 ** 5,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.1,
                'seed': seed,
                'nthread': 28,
                'n_jobs':24,
                'verbose': -1,
            }

            model = clf.train(params,    #训练模型
                      train_set=train_matrix, 
                      valid_sets=valid_matrix, 
                      num_boost_round=2000, 
                      verbose_eval=100, 
                      early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration) #预测，验证集预测
            test_pred = model.predict(test_x, num_iteration=model.best_iteration) #预测，测试集预测
            
        val_y=np.array(val_y).reshape(-1, 1) #验证集的标签值，从series类型转换成ndarray类型，reshape(-1, 1)，把数组变为一列，n*1
        val_y = onehot_encoder.fit_transform(val_y) #把验证集的标签值，改成哑编码格式
        print('预测的概率矩阵为：')
        print(test_pred)
        test += test_pred
        score=abs_sum(val_y, val_pred) # 每一折的 验证集的预测值与真实标签算出来的分数
        cv_scores.append(score)#把分数写入cv_score
        print(cv_scores)#打印分数
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    test=test/kf.n_splits

    return test #对测试集的预测值

In [25]:
def lgb_model(x_train, y_train, x_test):
    lgb_test = cv_model(lgb, x_train, y_train, x_test, "lgb") #训练模型
    return lgb_test

In [31]:
lgb_test = lgb_model(x_train, y_train, x_test) #测试集的预测值

************************************ 1 ************************************
[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=24. Current value: num_threads=24
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_logloss: 0.052514
[200]	valid_0's multi_logloss: 0.0387442
[300]	valid_0's multi_logloss: 0.0360959
[400]	valid_0's multi_logloss: 0.0367654
[500]	valid_0's multi_logloss: 0.0381283
Early stopping, best iteration is:
[324]	valid_0's multi_logloss: 0.0358704
预测的概率矩阵为：
[[9.99987167e-01 1.18712451e-05 1.98144017e-07 7.63742082e-07]
 [8.94060024e-06 4.05373475e-04 9.99585681e-01 4.48010105e-09]
 [6.80586216e-07 4.92127102e-08 1.68023344e-07 9.99999102e-01]
 ...
 [4.61991485e-03 1.94549335e-05 9.95360243e-01 3.87378343e-07]
 [9.99973622e-01 2.62969453e-05 5.79233235e-08 2.31103798e-08]
 [9.52202025e-01 1.27438478e-02 2.82481465e-02 6.80598064e-03]]
[272.2493733683497]
************************************ 2 *********

Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_logloss: 0.0547402
[200]	valid_0's multi_logloss: 0.0432049
[300]	valid_0's multi_logloss: 0.0406137
[400]	valid_0's multi_logloss: 0.0418444
[500]	valid_0's multi_logloss: 0.0441254
Early stopping, best iteration is:
[306]	valid_0's multi_logloss: 0.0404444
预测的概率矩阵为：
[[9.99921967e-01 7.64941533e-05 7.84133187e-07 7.55029571e-07]
 [4.63706455e-05 2.17797237e-04 9.99735825e-01 6.87231477e-09]
 [1.09590026e-06 6.76044955e-08 3.64721097e-07 9.99998472e-01]
 ...
 [3.97555550e-02 1.73212754e-04 9.60068936e-01 2.29615536e-06]
 [9.99898882e-01 1.01001966e-04 8.85561798e-08 2.77126943e-08]
 [9.63079579e-01 8.38062070e-03 2.65071922e-02 2.03260801e-03]]
[272.2493733683497, 352.2726129078004, 276.4128321031457, 279.2833654813336, 242.00338538277208, 290.2688429894397, 270.123458832031, 278.8380866463057, 297.85664276726374]
************************************ 10 ************************************
[LightGBM] [W

In [38]:
# lgb_test

In [39]:
temp=pd.DataFrame(lgb_test)

result=pd.read_csv('sample_submit.csv')
result['label_0']=temp[0]
result['label_1']=temp[1]
result['label_2']=temp[2]
result['label_3']=temp[3]
result.to_csv('submit.csv',index=False)